In [1]:
param_market = 'US'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                if param_market == 'US':
                    url = 'https://www.ebay.com/itm/' + input_.loc[i, 'Item_Number'] + '?_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'AU': 
                    url = 'https://www.ebay.com.au/itm/' + input_.loc[i, 'Item_Number'] + '?_ul=AU&_stpos=2019&orig_cvip=true'

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =
                
                js = json.loads(resp.split('''$MC=(window.$MC||[]).concat(''')[1].split(''']}]})</script></body></html>''')[0] + ']}]}')

                # = = = = = = = = = = = = = = =
                
                market_id = js['o']['g']['marketplaceId'].strip()

                # = = = = = = = = = = = = = = =
                
                country = js['o']['g']['country'].strip()

                # = = = = = = = = = = = = = = =
                
                language = js['o']['g']['language'].strip()

                # = = = = = = = = = = = = = = =
                
                locale = js['o']['g']['locale'].strip()

                # = = = = = = = = = = = = = = =
                
                item_id = js['o']['g']['itemId'].strip()

                # = = = = = = = = = = = = = = =
                
                if item_id != input_.loc[i, 'Item_Number']:
                    continue
                
                # = = = = = = = = = = = = = = =
                
                specification = js['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']

                # = = = = = = = = = = = = = = =
                
                if 'VEHICLE_SELECTION' in js['o']['w'][0][2]['model']['modules']:
                    category_id = js['o']['w'][0][2]['model']['modules']['VEHICLE_SELECTION']['callToActions'][0]['action']['params']['categoryId'].strip()
                    vehicle_page = js['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('of')[1].strip()
                else:
                    category_id = ''
                    vehicle_page = ''
                    
                # = = = = = = = = = = = = = = =
                
                title = js['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'subTitle' in js['o']['w'][0][2]['model']['modules']['TITLE']:
                    subtitle = js['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip()
                else:
                    subtitle = ''

                # = = = = = = = = = = = = = = =
                
                if 'binModel' in js['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = js['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
                elif 'bidPrice' in js['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = js['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()
                    
                # = = = = = = = = = = = = = = =
                
                if 'QUANTITY' in js['o']['w'][0][2]['model']['modules']:
                    available = ''
                    sold = ''
                    list_text = js['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans']
                    for text in list_text:
                        if 'available' in text['text']:
                            available = text['text'].strip()
                        elif 'sold' in text['text']:
                            sold = text['text'].strip()
                else:
                    available = ''
                    sold = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'largeImg' in js['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']:
                    src = js['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['largeImg']['URL'].strip()
                else:
                    src = js['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['originalImg']['URL'].strip()

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': '',
                                         'Item_Number': input_.loc[i, 'Item_Number'],
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Price': price,
                                         'Available': available,
                                         'Sold': sold,
                                         'Vehicle': '',
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src,
                                         'Vehicle_Page': vehicle_page,
                                         'Category_Id': category_id,
                                         'Market_Id': market_id,
                                         'Country': country,
                                         'Language': language,
                                         'Locale': locale}])

                for j, name_value in zip(range(len(specification)), specification):
                    label = specification[name_value]['labels'][0]['textSpans'][0]['text'].strip()
                    if j == 0 and label == 'Condition':
                        df_temp['Condition'] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip()
                    else:
                        df_temp[label.lower()] = specification[name_value]['values'][0]['textSpans'][0]['text'].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                     'No.': int(input_.loc[i, 'No.'])}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Item_Number'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True])
output_correct.to_excel('./item-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True])
    output_error.to_excel('./item_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：1301

186206141550  <->  [ok] - 剩余数量：1294 - 当前时间：08:53:44
174335830018  <->  [ok] - 剩余数量：1293 - 当前时间：08:53:44
185795035769  <->  [ok] - 剩余数量：1292 - 当前时间：08:53:44
175834987977  <->  [ok] - 剩余数量：1291 - 当前时间：08:53:44
175839141504  <->  [ok] - 剩余数量：1290 - 当前时间：08:53:44
175826724111  <->  [ok] - 剩余数量：1289 - 当前时间：08:53:45
175559174315  <->  [ok] - 剩余数量：1288 - 当前时间：08:53:46
174336549665  <->  [ok] - 剩余数量：1287 - 当前时间：08:53:47
175150164254  <->  [ok] - 剩余数量：1286 - 当前时间：08:53:47
185203099229  <->  [ok] - 剩余数量：1285 - 当前时间：08:53:47
186211660277  <->  [ok] - 剩余数量：1284 - 当前时间：08:53:48
186168985713  <->  [ok] - 剩余数量：1283 - 当前时间：08:53:49
175350823211  <->  [ok] - 剩余数量：1282 - 当前时间：08:53:49
174336451535  <->  [ok] - 剩余数量：1281 - 当前时间：08:53:49
186152963241  <->  [ok] - 剩余数量：1280 - 当前时间：08:53:49
175839135154  <->  [ok] - 剩余数量：1279 - 当前时间：08:53:50
185742446210  <->  [ok] - 剩余数量：1278 - 当前时间：08:53:51
186045197660  <->  [ok] - 剩余数量：1277 - 当前时间：08:53:51
186077708434  <->  [ok] - 剩余数量：1276 - 当前时间：08:53:51
18

175504504071  <->  [ok] - 剩余数量：1136 - 当前时间：08:55:26
185513193105  <->  [ok] - 剩余数量：1135 - 当前时间：08:55:27
186127140670  <->  [ok] - 剩余数量：1134 - 当前时间：08:55:27
185960686043  <->  [ok] - 剩余数量：1133 - 当前时间：08:55:28
175467750266  <->  [ok] - 剩余数量：1132 - 当前时间：08:55:28
185554505192  <->  [ok] - 剩余数量：1131 - 当前时间：08:55:29
175930024470  <->  [ok] - 剩余数量：1130 - 当前时间：08:55:30
175839139660  <->  [ok] - 剩余数量：1129 - 当前时间：08:55:30
184350081381  <->  [ok] - 剩余数量：1128 - 当前时间：08:55:32
175338385369  <->  [ok] - 剩余数量：1127 - 当前时间：08:55:33
185960682788  <->  [ok] - 剩余数量：1126 - 当前时间：08:55:33
184349362590  <->  [ok] - 剩余数量：1125 - 当前时间：08:55:33
175372768912  <->  [ok] - 剩余数量：1124 - 当前时间：08:55:34
174338542623  <->  [ok] - 剩余数量：1123 - 当前时间：08:55:34
173579751181  <->  [ok] - 剩余数量：1122 - 当前时间：08:55:35
174336907643  <->  [ok] - 剩余数量：1121 - 当前时间：08:55:35
183471987229  <->  [ok] - 剩余数量：1120 - 当前时间：08:55:36
174679105137  <->  [ok] - 剩余数量：1119 - 当前时间：08:55:38
175521154794  <->  [ok] - 剩余数量：1118 - 当前时间：08:55:38
175088907350

174212817286  <->  [ok] - 剩余数量：978 - 当前时间：08:58:52
175761374917  <->  [ok] - 剩余数量：977 - 当前时间：08:58:53
185466307488  <->  [ok] - 剩余数量：976 - 当前时间：08:58:54
185742433434  <->  [ok] - 剩余数量：975 - 当前时间：08:58:55
185365918672  <->  [ok] - 剩余数量：974 - 当前时间：08:58:55
185784583635  <->  [ok] - 剩余数量：973 - 当前时间：08:58:56
186011159237  <->  [ok] - 剩余数量：972 - 当前时间：08:58:56
174113708089  <->  [ok] - 剩余数量：971 - 当前时间：08:58:58
185246025107  <->  [ok] - 剩余数量：970 - 当前时间：08:58:59
175146867244  <->  [ok] - 剩余数量：969 - 当前时间：08:59:00
185690844112  <->  [ok] - 剩余数量：968 - 当前时间：08:59:01
186028682782  <->  [ok] - 剩余数量：967 - 当前时间：08:59:02
186116593481  <->  [ok] - 剩余数量：966 - 当前时间：08:59:03
175447826187  <->  [ok] - 剩余数量：965 - 当前时间：08:59:04
185743691498  <->  [ok] - 剩余数量：964 - 当前时间：08:59:04
175561794839  <->  [ok] - 剩余数量：963 - 当前时间：08:59:07
175171907447  <->  [ok] - 剩余数量：962 - 当前时间：08:59:09
176046559955  <->  [ok] - 剩余数量：961 - 当前时间：08:59:12
175559212356  <->  [ok] - 剩余数量：960 - 当前时间：08:59:13
185318300517  <->  [ok] - 剩余数量：

175900163193  <->  [ok] - 剩余数量：816 - 当前时间：09:02:03
186148477789  <->  [ok] - 剩余数量：815 - 当前时间：09:02:04
186014157351  <->  [ok] - 剩余数量：814 - 当前时间：09:02:05
184981836818  <->  [ok] - 剩余数量：813 - 当前时间：09:02:05
185996866599  <->  [ok] - 剩余数量：812 - 当前时间：09:02:06
175146861060  <->  [ok] - 剩余数量：811 - 当前时间：09:02:07
184880430375  <->  [ok] - 剩余数量：810 - 当前时间：09:02:07
186025872161  <->  [ok] - 剩余数量：809 - 当前时间：09:02:08
186207692465  <->  [ok] - 剩余数量：808 - 当前时间：09:02:08
175739858420  <->  [ok] - 剩余数量：807 - 当前时间：09:02:10
185685388660  <->  [ok] - 剩余数量：806 - 当前时间：09:02:11
176106899963  <->  [ok] - 剩余数量：805 - 当前时间：09:02:12
186207692939  <->  [ok] - 剩余数量：804 - 当前时间：09:02:12
174957823370  <->  [ok] - 剩余数量：803 - 当前时间：09:02:13
186306186595  <->  [ok] - 剩余数量：802 - 当前时间：09:02:15
186198784331  <->  [ok] - 剩余数量：801 - 当前时间：09:02:16
186168983757  <->  [ok] - 剩余数量：800 - 当前时间：09:02:18
184500772968  <->  [ok] - 剩余数量：799 - 当前时间：09:02:19
175498705539  <->  [ok] - 剩余数量：798 - 当前时间：09:02:19
175288246782  <->  [ok] - 剩余数量：

175077694191  <->  [ok] - 剩余数量：655 - 当前时间：09:05:17
184668164243  <->  [ok] - 剩余数量：654 - 当前时间：09:05:17
175511903318  <->  [ok] - 剩余数量：653 - 当前时间：09:05:17
186207690559  <->  [ok] - 剩余数量：652 - 当前时间：09:05:19
184162395595  <->  [ok] - 剩余数量：651 - 当前时间：09:05:20
175790760673  <->  [ok] - 剩余数量：650 - 当前时间：09:05:20
185578710052  <->  [ok] - 剩余数量：649 - 当前时间：09:05:20
185940679731  <->  [ok] - 剩余数量：648 - 当前时间：09:05:21
186304968205  <->  [ok] - 剩余数量：647 - 当前时间：09:05:22
175782590777  <->  [ok] - 剩余数量：646 - 当前时间：09:05:22
185571363381  <->  [ok] - 剩余数量：645 - 当前时间：09:05:23
175790764171  <->  [ok] - 剩余数量：644 - 当前时间：09:05:23
186114338896  <->  [ok] - 剩余数量：643 - 当前时间：09:05:24
175414461321  <->  [ok] - 剩余数量：642 - 当前时间：09:05:25
175906927825  <->  [ok] - 剩余数量：641 - 当前时间：09:05:26
184451423025  <->  [ok] - 剩余数量：640 - 当前时间：09:05:26
186006431165  <->  [ok] - 剩余数量：639 - 当前时间：09:05:27
175800393641  <->  [ok] - 剩余数量：638 - 当前时间：09:05:29
175633556590  <->  [ok] - 剩余数量：637 - 当前时间：09:05:29
175841504179  <->  [ok] - 剩余数量：

185315092870  <->  [ok] - 剩余数量：494 - 当前时间：09:08:19
186142069843  <->  [ok] - 剩余数量：493 - 当前时间：09:08:20
185803207379  <->  [ok] - 剩余数量：492 - 当前时间：09:08:21
175948886013  <->  [ok] - 剩余数量：491 - 当前时间：09:08:21
186111744796  <->  [ok] - 剩余数量：490 - 当前时间：09:08:22
185465259357  <->  [ok] - 剩余数量：489 - 当前时间：09:08:23
173828788221  <->  [ok] - 剩余数量：488 - 当前时间：09:08:24
174336632659  <->  [ok] - 剩余数量：487 - 当前时间：09:08:25
172396779158  <->  [ok] - 剩余数量：486 - 当前时间：09:08:25
186306186729  <->  [ok] - 剩余数量：485 - 当前时间：09:08:26
185969861660  <->  [ok] - 剩余数量：484 - 当前时间：09:08:26
175946691993  <->  [ok] - 剩余数量：483 - 当前时间：09:08:28
172327813853  <->  [ok] - 剩余数量：482 - 当前时间：09:08:28
175407502320  <->  [ok] - 剩余数量：481 - 当前时间：09:08:28
185592097712  <->  [ok] - 剩余数量：480 - 当前时间：09:08:31
174486874693  <->  [ok] - 剩余数量：479 - 当前时间：09:08:31
184600437897  <->  [ok] - 剩余数量：478 - 当前时间：09:08:31
175379140450  <->  [ok] - 剩余数量：477 - 当前时间：09:08:33
186152974142  <->  [ok] - 剩余数量：476 - 当前时间：09:08:34
176038232548  <->  [ok] - 剩余数量：

174378841005  <->  [ok] - 剩余数量：333 - 当前时间：09:12:02
185759949636  <->  [ok] - 剩余数量：332 - 当前时间：09:12:02
186110660912  <->  [ok] - 剩余数量：331 - 当前时间：09:12:03
175850619642  <->  [ok] - 剩余数量：330 - 当前时间：09:12:04
185991234356  <->  [ok] - 剩余数量：329 - 当前时间：09:12:04
185742434623  <->  [ok] - 剩余数量：328 - 当前时间：09:12:06
174546383658  <->  [ok] - 剩余数量：327 - 当前时间：09:12:06
182610645805  <->  [ok] - 剩余数量：326 - 当前时间：09:12:09
185183559385  <->  [ok] - 剩余数量：325 - 当前时间：09:12:09
175948891788  <->  [ok] - 剩余数量：324 - 当前时间：09:12:09
175834986322  <->  [ok] - 剩余数量：323 - 当前时间：09:12:11
186110651442  <->  [ok] - 剩余数量：322 - 当前时间：09:12:11
185459617250  <->  [ok] - 剩余数量：321 - 当前时间：09:12:12
175279974940  <->  [ok] - 剩余数量：320 - 当前时间：09:12:13
173264775465  <->  [ok] - 剩余数量：319 - 当前时间：09:12:13
185905918631  <->  [ok] - 剩余数量：318 - 当前时间：09:12:15
175660476287  <->  [ok] - 剩余数量：317 - 当前时间：09:12:16
173824384752  <->  [ok] - 剩余数量：316 - 当前时间：09:12:16
175789293458  <->  [ok] - 剩余数量：315 - 当前时间：09:12:18
186133886175  <->  [ok] - 剩余数量：

176084976215  <->  [ok] - 剩余数量：172 - 当前时间：09:14:34
185694584728  <->  [ok] - 剩余数量：171 - 当前时间：09:14:35
175816741084  <->  [ok] - 剩余数量：170 - 当前时间：09:14:36
175860832788  <->  [ok] - 剩余数量：169 - 当前时间：09:14:37
175045314966  <->  [ok] - 剩余数量：168 - 当前时间：09:14:38
175713688729  <->  [ok] - 剩余数量：167 - 当前时间：09:14:40
174211357023  <->  [ok] - 剩余数量：166 - 当前时间：09:14:41
185972680950  <->  [ok] - 剩余数量：165 - 当前时间：09:14:43
175442796548  <->  [ok] - 剩余数量：164 - 当前时间：09:14:44
186197166679  <->  [ok] - 剩余数量：163 - 当前时间：09:14:45
175712701309  <->  [ok] - 剩余数量：162 - 当前时间：09:14:47
175278518892  <->  [ok] - 剩余数量：161 - 当前时间：09:14:50
175297682970  <->  [ok] - 剩余数量：160 - 当前时间：09:14:50
175606122020  <->  [ok] - 剩余数量：159 - 当前时间：09:14:51
176244988016  <->  [ok] - 剩余数量：158 - 当前时间：09:14:52
175850601626  <->  [ok] - 剩余数量：157 - 当前时间：09:14:53
175297685987  <->  [ok] - 剩余数量：156 - 当前时间：09:14:55
175844367536  <->  [ok] - 剩余数量：155 - 当前时间：09:14:56
185676581969  <->  [ok] - 剩余数量：154 - 当前时间：09:14:57
185736030639  <->  [ok] - 剩余数量：

185969863190  <->  [ok] - 剩余数量：9 - 当前时间：09:18:28
184119267701  <->  [ok] - 剩余数量：8 - 当前时间：09:18:56
175304684074  <->  [ok] - 剩余数量：7 - 当前时间：09:19:25
185940676543  <->  [ok] - 剩余数量：6 - 当前时间：09:19:27
176036032294  <->  [ok] - 剩余数量：5 - 当前时间：09:19:29
185899656792  <->  [ok] - 剩余数量：4 - 当前时间：09:19:31
185690830792  <->  [ok] - 剩余数量：3 - 当前时间：09:19:34
185785918410  <->  [ok] - 剩余数量：2 - 当前时间：09:19:36
175755903104  <->  [ok] - 剩余数量：1 - 当前时间：09:19:39
175616508614  <->  [ok] - 剩余数量：0 - 当前时间：09:19:42
185331363409  <->  [ok] - 剩余数量：0 - 当前时间：09:19:46
175426818511  <->  [ok] - 剩余数量：0 - 当前时间：09:19:51
175803440345  <->  [ok] - 剩余数量：0 - 当前时间：09:21:17
176226048742  <->  [ok] - 剩余数量：0 - 当前时间：09:21:40
185980125403  <->  [ok] - 剩余数量：0 - 当前时间：09:21:58
185940650858  <->  [ok] - 剩余数量：0 - 当前时间：09:23:07

爬虫输出中

爬虫结束
